# Using the Simulators

The bmtk is capable of running a number of different neuronal simulators across various levels of resolutions. 

BioNet - Biophysical neuronal simulation using BioNet  
PointNet - Point neuron simulation using NEST  
FilterNet - Filtering based models using LGNModel  
PopNet - Population rate models using DiPDE  
<s>MintNet - Convolutional neural nets with TensorFlow</s>

Different simulators will have different features and will require different parameters and options to run. But the bmtk uses the same framework for running the different types of simulators and producing output - utilizing the SONATA framework. The following will in general describe how to go about setting up and running one of the various simulators. 

## Building a network

A main philosophy of the bmtk is to separate the processes of building a network model and running a simulation versus other simulators which use the same script to build a network and then immedietly run a simulation. The main advantages of doing so is so that:
* Different parts of a network can be built independently by different teams.
* Allows models to be shared.
* Models can be readily ported from one simulator to another.
* Faster to run a large pre-built network across multiple simulations without having to recalculate the connection matricies every time.
* Easier to adjust and update parameters between simulations.

The bmtk uses the SONATA format to store networks. See the [previous tutorial for building networks](NetworkBuilder_Intro.ipynb) for more information.

## Setting up a configuration file

To create the a configuration file and simulation enviornment you can either download and copy a simulator [environment template directory](../environments). Alternatively you can build it using the following:

```bash
$ python -m bmtk.utils.sim_setup -n network_dir/ <options> [bionet|pointnet|popnet|...]
```

All the parameters to setup and run a simulation is stored in the file *config.json*. More detail about the configuration file can be found in the [sonata documentation](https://github.com/AllenInstitute/sonata/blob/master/docs/SONATA_DEVELOPER_GUIDE.md#tying-it-all-together---the-networkcircuit-config-file), but we will provide a brief overview of the various sections:

### The mainfest

```json
"manifest": {
    "$BASE_DIR": "${configdir}",
    "$OUTPUT_DIR": "$BASE_DIR/output",
    "$INPUT_DIR": "$BASE_DIR/../nwb_files",
    ...
},
```

An optional section that allows users to create variables and references that can be used in other parts of the config.


### Run-time and conditions
```json
"run": {
    "tstop": 3000.0,
    "dt": 0.1,
    "spike_threshold": -15,
    "nsteps_block": 5000,
    "dL": 20.0
},

"conditions": {
    "celsius": 34.0,
    "v_init": -80
},
```

Parameters for how long a simulation will run (in milliseconds) and other simulation parameters and conditions - depending on the type of simulator.

### Components
```json
"components": {
    "morphologies_dir": "$COMPONENT_DIR/morphologies",
    "synaptic_models_dir": "$COMPONENT_DIR/synaptic_models",
    "mechanisms_dir":"$COMPONENT_DIR/mechanisms",
    "biophysical_neuron_models_dir": "$COMPONENT_DIR/biophysical_neuron_templates",
    "point_neuron_models_dir": "$COMPONENT_DIR/point_neuron_templates"
},
```

List of directories contains model files for individual cells and synapses.

### Inputs

```json
"inputs": {
    "spike_trains": {
        "input_type": "spikes",
        "module": "nwb",
        "input_file": "$INPUT_DIR/lgn_spikes.nwb",
        "node_set": "lgn"
    },
 	"current_clamp": {
        "input_type": "current_clamp",
        "module": "IClamp",
        "node_set": [0, 1, 2, 3, 4],
        "amp": 200.1178,
        "delay": 100.0,
        "duration": 2000.0
    }
},
``` 

Specifying the types of inputs the network will receive during the simulation. Including spike trains/rates for virtual or external cells, current clamp stimulation, extracellular stimulation, etc.

### Output and reports

```json
"output":{ 
    "log_file": "log.txt",
    "output_dir": "$OUTPUT_DIR",
    "spikes_file": "spikes.h5",
    "overwrite_output_dir": true
},
"reports": {
    "calcium_concentration": {
        "cells": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        "variable_name": "cai",
        "module": "membrane_report",
        "file_name": "cell_vars.h5",
        "sections": "soma"
    },
    "ecp": {
        "cells": [0],
        "variable_name": "v",
        "module": "extracellular",
        "electrode_positions": "$COMPONENT_DIR/recXelectrodes/linear_electrode.csv",
        "ecp_file": "ecp.h5",
        "electrode_channels": "all",
        "contributions_dir": "ecp_contributions"
    }
},
```

The output section is used to specify the main directory where output is written to, as well as log file location and default outputs. Unless absolute paths are specified any file created during the simulation will be written into the directory specified in "output/output_dir".

The reports allows user to specify more advanced types of simulation reporting like, depending on the simulator, calcium concentration or local field potentials

### Networks
```json
"networks": {
    "nodes": [
    {
        "nodes_file": "$NETWORK_DIR/v1_nodes.h5",
        "node_types_file": "$NETWORK_DIR/v1_node_types.csv"
    },
    {
        "nodes_file": "$NETWORK_DIR/lgn_nodes.h5",
        "node_types_file": "$NETWORK_DIR/lgn_node_types.csv"
    }
    ],

    "edges": [
    {
        "edges_file": "$NETWORK_DIR/v1_v1_edges.h5",
        "edge_types_file": "$NETWORK_DIR/v1_v1_edge_types.csv"
    },
    {
        "edges_file": "$NETWORK_DIR/lgn_v1_edges.h5",
        "edge_types_file": "$NETWORK_DIR/lgn_v1_edge_types.csv"
    }
    ]
}
```

Section to specify the different nodes and edges files used during the simulation

## Running the simulation file

Once you have setup your configuration and put the network and model files in their correct places the running a simulation can be done without any extra programming. 

```bash
$ python run_simulator.py config.json
```

The simulation will run and save the results in the files specified in the output and reports sections of the config. The bmtk also includes a small but growing list of tools for analyzing and graphing the output describe in future chapters.

The bmtk also allows for users to develop custom python code that will executed during the simulation runtime. Including code to tell the simulator to load customized cell and synaptic models, or how to adjust connection weights before and during simulations. Theses options will vary from simulator to simulator and will be futher describe in each simulators documention.

#### Running in Parallel mode

Some of the simulation engines, BioNet and PointNet, allow you to run a network across multple machines/nodes using MPI, but typically require slightly different commands:

#### BioNet (NEURON)
```bash
$ mpirun -n $NNODES nrniv nrniv -mpi -python run_bionet.py config.json
```

#### PointNet (NEST)
```bash
$ mpirun -n $NNODES python run_pointnet.py config.json
```

If you're running the simulations on a HPC cluster make sure to follow the instructions as required by the workload scheduler being used.